# Simple chat with docs

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import ipywidgets as widgets
from IPython.display import display

In [ ]:
local_model = False # todo: the following does't work.
if local_model:
    # llama-index-embeddings-huggingface llama-index-llms-ollama
    from llama_index.core import Settings
    from llama_index.embeddings.huggingface import HuggingFaceEmbedding
    from llama_index.llms.ollama import Ollama
    
    # bge-base embedding model
    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
    
    # ollama
    Settings.llm = Ollama(model="llama3", request_timeout=360.0)

In [ ]:
class Chat():
    def __init__(self):
        self.former_folder = None
        
    def load(self, folder):
        if self.former_folder == folder:
            return len(self.documents)
        self.former_folder = folder
        
        documents = SimpleDirectoryReader(folder).load_data()
        index = VectorStoreIndex.from_documents(documents)
        self.query_engine = index.as_query_engine()
        self.documents = documents
        return len(documents)
        
    def query(self, question):
        return self.query_engine.query(question)

In [ ]:
# Create widgets
output_label = widgets.HTML(value="")

docs_input = widgets.Text(value="docs", placeholder="Enter a directory here")
load_button = widgets.Button(description="Load")

text_input = widgets.Text(value="What is the document about?", placeholder="Enter a question here")
submit_button = widgets.Button(description="Submit")

chat = Chat()

def on_load(e):    
    number_of_documents = chat.load(docs_input.value)
    output_label.value = f"""
    <div style='text-align:left; color: darkgrey; font-size: 20px'>{number_of_documents} documents loaded.</div>
    """

def on_submit(e):
    question = text_input.value
    if len(question) == 0:
        return
    text_input.value = ""

    # submit prompt to LLM
    answer = chat.query(question)

    # Append question and answer to the existing HTML content
    output_label.value += f"""
    <div style='text-align:right; color: blue; font-size: 20px'>{question}</div>
    <div style='text-align:left; color: darkgreen; font-size: 20px'>{answer}</div>
    """

# Attach the event handler to the text field and the button
docs_input.continuous_update = False
docs_input.observe(on_load)
load_button.on_click(on_load)

text_input.continuous_update = False
text_input.observe(on_submit)
submit_button.on_click(on_submit)

# Arrange the widgets for display
display(widgets.HBox([docs_input, load_button]), output_label, widgets.HBox([text_input, submit_button]))